In [59]:
# Deps
from PIL import Image
import random

import numpy as np

import torch
from torch import nn
from torchvision import models, transforms

In [86]:
file_path = "./data/regionsnorth/data/Aitekebiiskii_evw/3676/3676_2020-05-12.png"

In [87]:
# Lables

labels = sorted(["nothing", "unvisible", "sowing", "harvesting"])
label_to_idx = {cls_name: i for i, cls_name in enumerate(labels)}
idx_to_label = {value:key for key, value in label_to_idx.items()}

In [88]:
# Transform def
h, w =224, 224
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
        transforms.Resize((h,w)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
])

In [89]:
# Load file and transform
frame = Image.open(file_path).convert('RGB')

seed = np.random.randint(1e9)

random.seed(seed)
np.random.seed(seed)

frame = transform(frame)

In [90]:
# Configuring device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [94]:
# Loading model
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(labels))
model = model.to(device)

model.load_state_dict(torch.load("./models/weights_classification.pt", map_location=device))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [95]:
# Running model
with torch.no_grad():
    output = model(frame.unsqueeze(0).to(device))
    _, preds = torch.max(output, 1)
           
    outputs = torch.sigmoid(output)
    outputs[outputs < 0.5] = 0
    
    print(idx_to_label)
    print(outputs)

{0: 'harvesting', 1: 'nothing', 2: 'sowing', 3: 'unvisible'}
tensor([[0.0000, 0.6602, 0.9947, 0.0000]], device='cuda:0')


In [110]:
# Result
predictions = outputs.cpu().numpy()[0]
for label in labels:
    print("{0}: {1:.2f}".format(label, predictions[label_to_idx[label]]))

harvesting: 0.00
nothing: 0.66
sowing: 0.99
unvisible: 0.00
